# Band Structure at Fixed Quasimomentum - Two Parameter Scan
This notebook scans TWO lattice parameters at a fixed quasimomentum q_in = [0., 0.] (Gamma point)

*This is directly from Perplexity and won't work without edits. It is probably better actually to start with my own code and copy things from here.*

In [1]:
# import all the E9 stuff
import logging
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import time
from scipy.linalg import eigh
from collections import defaultdict

# User defined modules
E9path = Path("C:/", "Users", "ken92", "Documents", "Studies", "E5", "simulation", "E9_simulations")
if str(E9path) not in sys.path:
    sys.path.insert(1, str(E9path))
import E9_fn.E9_constants as E9c
import E9_fn.E9_atom as E9a
import E9_fn.E9_cooltrap as E9ct
import E9_fn.plane_wave_expansion.blochstate_class as bsc
from E9_fn import util

# Logging
logpath = ''
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename=logpath, level=loglevel)

c:\Users\ken92\Documents\Studies\E5\simulation\E9_simulations\.venv\Lib\site-packages\gftool\precision.py:16: UserWarning: No quad precision datatypes available!
Some functions might be less accurate.
  warnings.warn("No quad precision datatypes available!\n"


# Inputs

In [2]:
species = "K40"  # "Rb87", "K40"

# Fixed quasimomentum (Gamma point)
q_in = np.array([0., 0.])

# ===== CHOOSE YOUR TWO PARAMETERS TO SCAN =====
# Options: 'V1064Er', 'V532Er', 'phi12', 'phi23', 
#          'B1_rel_int_1064', 'B1_rel_int_532', 'B3_rel_int_1064', 'B3_rel_int_532'
scan_param1 = 'V1064Er'
scan_param2 = 'V532Er'

# Define the ranges to scan
# Parameter 1
if scan_param1 == 'V1064Er':
    param1_values = np.linspace(4., 12., 20)  # Reduce points for 2D grid (quadratic scaling)
elif scan_param1 == 'V532Er':
    param1_values = np.linspace(0.5, 4., 20)
elif scan_param1 in ['phi12', 'phi23']:
    param1_values = np.linspace(0., 2*np.pi, 20)
elif scan_param1 in ['B1_rel_int_1064', 'B1_rel_int_532', 'B3_rel_int_1064', 'B3_rel_int_532']:
    param1_values = np.linspace(0., 1.5, 20)
else:
    raise ValueError(f"Unknown parameter: {scan_param1}")

# Parameter 2
if scan_param2 == 'V1064Er':
    param2_values = np.linspace(4., 12., 20)
elif scan_param2 == 'V532Er':
    param2_values = np.linspace(0.5, 4., 20)
elif scan_param2 in ['phi12', 'phi23']:
    param2_values = np.linspace(0., 2*np.pi, 20)
elif scan_param2 in ['B1_rel_int_1064', 'B1_rel_int_532', 'B3_rel_int_1064', 'B3_rel_int_532']:
    param2_values = np.linspace(0., 1.5, 20)
else:
    raise ValueError(f"Unknown parameter: {scan_param2}")

# Fixed parameters (defaults)
V1064Er_default = 8.0
V532Er_default = 2.0
phi12_default = 0.0
phi23_default = 0.0
B1_rel_int_1064_default = 1.0
B1_rel_int_532_default = 1.0
B3_rel_int_1064_default = 1.0
B3_rel_int_532_default = 1.0
n0nom_default = 0
ABoffset1064nom_default = 0

# Basic simulation parameters
num = 6  # size of q-momentum space
bandstart = 0  # starting band
bandend = 5  # ending band (inclusive)
save_results = False

# Initialization

In [3]:
if species == "Rb87":
    all_units_dict = E9c.all_lat_unit_Rb87
    Er_1064 = E9c.E_r1064_Rb87
    Er_532 = E9c.E_r532_Rb87
elif species == "K40":
    all_units_dict = E9c.all_lat_unit_K40
    Er_1064 = E9c.E_r1064_K40
    Er_532 = E9c.E_r532_K40
else:
    raise ValueError("Unknown species: {}".format(species))

m_unit = all_units_dict["m_unit"]
l_unit = all_units_dict["l_unit"]
E_unit = all_units_dict["E_unit"]
f_unit = all_units_dict["f_unit"]
t_unit = all_units_dict["t_unit"]

In [4]:
# Set up Hamiltonian components (these don't change during the scan)
k_center = (0, 0)
bandnum = bandend - bandstart + 1
size = 2 * num + 1

# Pre-compute Hamiltonian matrix components
Hq_mmat, Hq_nmat, H_532_template, H_1064_template = bsc.find_H_components(num, {'species': species}, center=k_center)

print(f"Scanning {scan_param1} and {scan_param2}")
print(f"{scan_param1} range: {param1_values[0]:.4f} to {param1_values[-1]:.4f} ({len(param1_values)} points)")
print(f"{scan_param2} range: {param2_values[0]:.4f} to {param2_values[-1]:.4f} ({len(param2_values)} points)")
print(f"Total grid points: {len(param1_values) * len(param2_values)}")

KeyError: 'V1064'

# 2D Scan and Calculate Band Energies

In [ ]:
# Storage for results - 3D array: (param1, param2, band)
band_energies_2d = np.zeros((len(param1_values), len(param2_values), bandnum))

start_time = time.time()

total_points = len(param1_values) * len(param2_values)
count = 0

for i, param1_val in enumerate(param1_values):
    for j, param2_val in enumerate(param2_values):
        # Set parameters - start with defaults
        V1064Er = V1064Er_default
        V532Er = V532Er_default
        phi12 = phi12_default
        phi23 = phi23_default
        B1_rel_int_1064 = B1_rel_int_1064_default
        B1_rel_int_532 = B1_rel_int_532_default
        B3_rel_int_1064 = B3_rel_int_1064_default
        B3_rel_int_532 = B3_rel_int_532_default
        n0nom = n0nom_default
        ABoffset1064nom = ABoffset1064nom_default
        
        # Helper function to set parameter
        def set_param(param_name, value):
            nonlocal V1064Er, V532Er, phi12, phi23, B1_rel_int_1064, B1_rel_int_532, B3_rel_int_1064, B3_rel_int_532
            if param_name == 'V1064Er':
                V1064Er = value
            elif param_name == 'V532Er':
                V532Er = value
            elif param_name == 'phi12':
                phi12 = value
            elif param_name == 'phi23':
                phi23 = value
            elif param_name == 'B1_rel_int_1064':
                B1_rel_int_1064 = value
            elif param_name == 'B1_rel_int_532':
                B1_rel_int_532 = value
            elif param_name == 'B3_rel_int_1064':
                B3_rel_int_1064 = value
            elif param_name == 'B3_rel_int_532':
                B3_rel_int_532 = value
        
        # Override the scanned parameters
        set_param(scan_param1, param1_val)
        set_param(scan_param2, param2_val)
        
        # Compute lattice parameters
        V532nom = V532Er * Er_532 / E9c.hnobar / 1e3
        V1064nom = V1064Er * Er_1064 / E9c.hnobar / 1e3
        V532 = 2 * np.pi * V532nom * 1e3 / f_unit
        V1064 = 2 * np.pi * V1064nom * 1e3 / f_unit
        ABoffset1064 = 2 * np.pi * ABoffset1064nom * 1e3 / f_unit
        n0 = n0nom * l_unit**3
        
        # Build experiment dictionary
        Exp_lib = {"species": species, "units_dict": all_units_dict,
                   'V532nom': V532nom, 'V1064nom': V1064nom, 'V532': V532, 'V1064': V1064,
                   'B1_rel_int_532': B1_rel_int_532, 'B1_rel_int_1064': B1_rel_int_1064,
                   'B3_rel_int_532': B3_rel_int_532, 'B3_rel_int_1064': B3_rel_int_1064,
                   'n0nom': n0nom, 'n0': n0,
                   'ABoffset1064nom': ABoffset1064nom, 'ABoffset1064': ABoffset1064,
                   'phi12': phi12, 'phi23': phi23}
        
        # Calculate Hamiltonian at q_in and diagonalize
        H = bsc.find_H(q_in, Exp_lib, Hq_mmat, Hq_nmat, H_532_template, H_1064_template)
        evals, evecs = eigh(H, eigvals=(bandstart, bandend), overwrite_a=True, check_finite=False)
        
        band_energies_2d[i, j, :] = np.real(evals)
        
        count += 1
        if count % max(1, total_points // 10) == 0:
            print(f"  Processed {count}/{total_points} points")

elapsed_time = time.time() - start_time
print(f"\n--- {elapsed_time:.2f} seconds ---")

# Results and Plots

In [ ]:
# Convert energies to kHz
band_energies_2d_kHz = band_energies_2d * f_unit / 1e3 / (2 * np.pi)

# Create contour plots for each band
n_cols = 3
n_rows = (bandnum + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
axes = axes.flatten() if bandnum > 1 else [axes]

for band_idx in range(bandnum):
    ax = axes[band_idx]
    energy_data = band_energies_2d_kHz[:, :, band_idx]
    
    im = ax.contourf(param2_values, param1_values, energy_data, levels=20, cmap='viridis')
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Energy (kHz)')
    
    ax.set_xlabel(scan_param2, fontsize=11)
    ax.set_ylabel(scan_param1, fontsize=11)
    ax.set_title(f'Band {band_idx + bandstart} Energy', fontsize=12)

# Remove extra subplots
for idx in range(bandnum, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Plot band gaps (difference between consecutive bands)
n_gaps = bandnum - 1
n_cols = 3
n_rows = (n_gaps + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
axes = axes.flatten() if n_gaps > 1 else [axes]

for gap_idx in range(n_gaps):
    ax = axes[gap_idx]
    gap_data = band_energies_2d_kHz[:, :, gap_idx + 1] - band_energies_2d_kHz[:, :, gap_idx]
    
    im = ax.contourf(param2_values, param1_values, gap_data, levels=20, cmap='RdYlGn')
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Gap (kHz)')
    
    ax.set_xlabel(scan_param2, fontsize=11)
    ax.set_ylabel(scan_param1, fontsize=11)
    ax.set_title(f'Gap (Band {gap_idx + bandstart} - Band {gap_idx + bandstart + 1})', fontsize=12)

# Remove extra subplots
for idx in range(n_gaps, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

In [ ]:
# Summary: Find minimum band gap across the 2D grid
print("\n=== SUMMARY ===")
print(f"Scanned parameters: {scan_param1} and {scan_param2}")
print(f"Fixed quasimomentum: q = {q_in}\n")

for gap_idx in range(bandnum - 1):
    gap_data = band_energies_2d_kHz[:, :, gap_idx + 1] - band_energies_2d_kHz[:, :, gap_idx]
    min_gap_idx = np.unravel_index(np.argmin(gap_data), gap_data.shape)
    min_gap_value = gap_data[min_gap_idx]
    min_param1 = param1_values[min_gap_idx[0]]
    min_param2 = param2_values[min_gap_idx[1]]
    
    print(f"Gap {gap_idx} (Band {gap_idx + bandstart} - Band {gap_idx + bandstart + 1}):")
    print(f"  Minimum gap: {min_gap_value:.4f} kHz")
    print(f"  at {scan_param1} = {min_param1:.4f}, {scan_param2} = {min_param2:.4f}")
    print()

In [ ]:
# Optional: 1D slices through the 2D grid for inspection
# Plot for a fixed value of param2 (middle value)
mid_idx = len(param2_values) // 2
fig, ax = plt.subplots(figsize=(10, 6))

for band_idx in range(bandnum):
    ax.plot(param1_values, band_energies_2d_kHz[:, mid_idx, band_idx], 
            label=f'Band {band_idx + bandstart}', marker='o', markersize=3)

ax.set_xlabel(scan_param1, fontsize=12)
ax.set_ylabel('Energy (kHz)', fontsize=12)
ax.set_title(f'1D Slice: {scan_param1} scan at {scan_param2} = {param2_values[mid_idx]:.4f}', fontsize=14)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()